## Dropout
- 학습 과정에서 신경망의 일부를 고의로 누락(Drop)시키는 것
- 특정 뉴런이나 Feature에 모델이 너무 의존하지 못하게 강제
- Train에는 p 만큼 끄고, 실전때에는 전부 다 사용

- Inverted Droptout
    1. Mask 생성
        - 각 뉴런이 살아남을지(1), 죽을지(0) 결정하는 마스크 벡터 m 생성
        $$m \sim \text{Bernoulli}(1 - p)$$
        - $1-p$: 생존할 확률 (Keep probability)
    1. 적용 및 스케일링
        - 입력값에 마스크를 곱하고, 살아남은 값들의 크기를 키워줌
        $$y = \frac{1}{1-p} (x \cdot m)$$
        - $x \cdot m$: 마스크를 씌워서 일부를 0으로 만듦
        - $\frac{1}{1-p}$: 스케일링 팩터 (Scaling Factor)

In [1]:
import torch
import torch.nn as nn

In [4]:
dropout = nn.Dropout(p=0.7)

tensor = torch.ones((3,5))

print(tensor)

print(dropout(tensor))

tensor([[1., 1., 1., 1., 1.],
        [1., 1., 1., 1., 1.],
        [1., 1., 1., 1., 1.]])
tensor([[0.0000, 3.3333, 0.0000, 0.0000, 3.3333],
        [0.0000, 3.3333, 3.3333, 0.0000, 0.0000],
        [0.0000, 0.0000, 0.0000, 3.3333, 0.0000]])


In [ ]:
tensor = torch.tensor([2, 4, 6, 8])
mask = torch.tensor([1,1,0,1])

new = tensor * mask
p=0.25
new * 1 / (1-p)


tensor([ 2.6667,  5.3333,  0.0000, 10.6667])

In [14]:
# 베르누이 분포 생성
p = 0.3
keep_p = 1-p

x = torch.randn(4,5)
probs = torch.full_like(x, keep_p)
print(probs)


mask = torch.bernoulli(probs)
print(mask)


tensor([[0.7000, 0.7000, 0.7000, 0.7000, 0.7000],
        [0.7000, 0.7000, 0.7000, 0.7000, 0.7000],
        [0.7000, 0.7000, 0.7000, 0.7000, 0.7000],
        [0.7000, 0.7000, 0.7000, 0.7000, 0.7000]])
tensor([[1., 1., 0., 1., 0.],
        [1., 1., 0., 1., 0.],
        [1., 1., 1., 1., 0.],
        [1., 1., 1., 1., 1.]])


In [24]:
# Uniform 분포에서 0~1 사이의 숫자를 생성
x = torch.randn(4,5)
p = 0.3
keep_p = 1-p

random_tensor = torch.rand_like(x)
print(random_tensor)

# keep_p보다 크면 T, 아니면 F
mask = random_tensor < keep_p
print(mask)

print(x * mask)

print(x * mask / keep_p)


tensor([[0.5083, 0.5860, 0.3241, 0.4371, 0.8256],
        [0.6275, 0.7895, 0.3189, 0.2753, 0.2967],
        [0.6713, 0.2147, 0.0625, 0.9536, 0.1458],
        [0.3536, 0.3130, 0.2128, 0.4740, 0.2092]])
tensor([[ True,  True,  True,  True, False],
        [ True, False,  True,  True,  True],
        [ True,  True,  True, False,  True],
        [ True,  True,  True,  True,  True]])
tensor([[-2.5854,  0.2114, -1.6395,  1.0170,  0.0000],
        [-0.0690, -0.0000,  1.5139,  1.4819, -0.4544],
        [-1.4147, -2.0630,  0.2821,  0.0000, -0.9749],
        [-0.7075, -1.2863, -0.5846, -0.8538,  0.3314]])
tensor([[-3.6934,  0.3020, -2.3422,  1.4529,  0.0000],
        [-0.0986, -0.0000,  2.1627,  2.1170, -0.6492],
        [-2.0210, -2.9472,  0.4030,  0.0000, -1.3927],
        [-1.0106, -1.8376, -0.8352, -1.2197,  0.4735]])


In [26]:
class Dropout:
    def __init__(
            self,
            p: float
    ):  
        if not (0.0 <= p < 1.0):
            raise ValueError("p must be in [0,1)")
        self.p = p
        self.training = True

    def __call__(self, x: torch.Tensor) -> torch.Tensor:
        return self.forward(x)

    def forward(self, x: torch.Tensor) -> torch.Tensor:
        if not self.training or self.p == 0.0:
            return x
        
        keep_prob = 1.0 - self.p
        mask = torch.rand_like(x) < keep_prob
        
        return (x * mask) / keep_prob

    def train(self, mode: bool=True):
        self.training = mode
        return self
    
    def eval(self):
        self.training = False
        return self.train(False)

In [31]:
torch.manual_seed(0)
x = torch.randn(4,5)

p = 0.4
my = Dropout(p).eval()
test = nn.Dropout(p).eval()

y_my = my(x)
y_test = test(x)

assert torch.equal(y_my, x)
assert torch.equal(y_test, x)
assert torch.equal(y_my, y_test)